# Create ISA-API Investigation from Datascriptor Study Design configuration
# Observational study on Human students

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 4-arm study design. Each arm has 10 subjects
- subjects are human, male and female, of three age groups in years (6-10, 10-15, 15-18) 
- two phases: the observational phase and follow-up
- one sample type colllected: blood
- two assay types: 


## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

In [2]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [3]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [4]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [5]:
with open(os.path.abspath(os.path.join(
    "config", "observational-study-students.json"
)), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'_id': '606454b9d071027feb44ad9b',
 'name': "Student's study",
 'description': 'In an observational study, the investigator simply records observations and analyzes data, without administration of an intervention or alteration of the care people receive. These studies may focus on risk factors, natural history, variations in disease progression or disease treatment without delivering an intervention. They often assess specific health characteristics of the enrolled human subjects by collecting medical/dental history, exposure, or clinical data; obtaining biospecimens (e.g., for biomarker or genomic analyses); or obtaining photographic, radiographic or other images from research subjects.',
 'design': {'isPending': False,
  'subjectType': 'Homo sapiens',
  'subjectSize': 100,
  'designType': {'term': 'observational design',
   'id': 'NCIT:C147138',
   'iri': 'http://purl.obolibrary.org/obo/NCIT_C147138',
   'label': 'Study subjects do not receive treatments',
   'value': 'observational

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design()` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [6]:
study_design = generate_study_design(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [7]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study from the design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(identifier='inv01', studies=[study])

The generation of the study from the design took 3.94 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [8]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 2.11 s.


In [9]:
directory = os.path.abspath(os.path.join('output', 'observational-design-students'))
os.makedirs(directory, exist_ok=True)
with open(os.path.abspath(os.path.join(directory, 'isa-investigation-observational-design-students.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [10]:
start = time()
isatab.dump(investigation, directory)
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The Tab serialisation of the ISA investigation took 66.29 s.


To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [11]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [12]:
len(dataframes)

3

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [13]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'

'a_AT11_clinical-chemistry_marker-panel.txt'

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the six arms for a total of 60 subjects. 5 blood samples per subject are collected (1 in treatment 1 phase, 1 in treatment, and 3 in the follow-up phase) for a total of 300 blood samples. These will undergo the NMR assay. We have 4 saliva samples per subject (1 during screen and 3 during follow-up) for a total of 240 saliva samples. These will undergo the "mass spcetrometry" assay.

In [20]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm1_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
count_arm4_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP4' in el)])
count_arm5_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP5' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP1 arm (i.e. group)".format(count_arm1_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))
print("There are {} samples in the GRP4 arm (i.e. group)".format(count_arm4_samples))
print("There are {} samples in the GRP5 arm (i.e. group)".format(count_arm5_samples))

There are 400 samples in the GRP0 arm (i.e. group)
There are 400 samples in the GRP1 arm (i.e. group)
There are 400 samples in the GRP2 arm (i.e. group)
There are 400 samples in the GRP3 arm (i.e. group)
There are 400 samples in the GRP4 arm (i.e. group)
There are 400 samples in the GRP5 arm (i.e. group)


In [21]:
study_frame

,Source Name,Characteristics[Study Subject],Protocol REF,Parameter Value[Sampling order],Parameter Value[Study cell],Date,Performer,Sample Name,Characteristics[organism part],Comment[study step with treatment],Factor Value[Sequence Order],Factor Value[DURATION],Unit
0,GRP0_SBJ001,Homo sapiens,sample collection,343,A0E1,2021-06-29,Unknown,GRP0_SBJ001_A0E1_SMP-blood-sample-1,blood sample,NO,1,300,days
1,GRP0_SBJ001,Homo sapiens,sample collection,127,A0E0,2021-06-29,Unknown,GRP0_SBJ001_A0E0_SMP-blood-sample-1,blood sample,NO,0,180,days
2,GRP0_SBJ001,Homo sapiens,sample collection,129,A0E0,2021-06-29,Unknown,GRP0_SBJ001_A0E0_SMP-blood-sample-3,blood sample,NO,0,180,days
3,GRP0_SBJ001,Homo sapiens,sample collection,128,A0E0,2021-06-29,Unknown,GRP0_SBJ001_A0E0_SMP-blood-sample-2,blood sample,NO,0,180,days
4,GRP0_SBJ002,Homo sapiens,sample collection,280,A0E0,2021-06-29,Unknown,GRP0_SBJ002_A0E0_SMP-blood-sample-1,blood sample,NO,0,180,days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,GRP5_SBJ099,Homo sapiens,sample collection,2321,A5E1,2021-06-29,Unknown,GRP5_SBJ099_A5E1_SMP-blood-sample-1,blood sample,NO,1,300,days
2396,GRP5_SBJ100,Homo sapiens,sample collection,2007,A5E0,2021-06-29,Unknown,GRP5_SBJ100_A5E0_SMP-blood-sample-1,blood sample,NO,0,180,days
2397,GRP5_SBJ100,Homo sapiens,sample collection,2009,A5E0,2021-06-29,Unknown,GRP5_SBJ100_A5E0_SMP-blood-sample-3,blood sample,NO,0,180,days
2398,GRP5_SBJ100,Homo sapiens,sample collection,2303,A5E1,2021-06-29,Unknown,GRP5_SBJ100_A5E1_SMP-blood-sample-1,blood sample,NO,1,300,days


In [22]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt']

,Sample Name,Comment[study step with treatment],Protocol REF,Performer,Raw Data File
0,GRP0_SBJ001_A0E0_SMP-blood-sample-1,NO,sample preparation,Unknown,AT11-S254-raw_data_file-R1
1,GRP0_SBJ001_A0E0_SMP-blood-sample-1,NO,sample preparation,Unknown,AT11-S253-raw_data_file-R1
2,GRP0_SBJ001_A0E0_SMP-blood-sample-2,NO,sample preparation,Unknown,AT11-S256-raw_data_file-R1
3,GRP0_SBJ001_A0E0_SMP-blood-sample-2,NO,sample preparation,Unknown,AT11-S255-raw_data_file-R1
4,GRP0_SBJ001_A0E0_SMP-blood-sample-3,NO,sample preparation,Unknown,AT11-S257-raw_data_file-R1
...,...,...,...,...,...
3995,GRP4_SBJ100_A4E0_SMP-blood-sample-2,NO,sample preparation,Unknown,AT11-S3419-raw_data_file-R1
3996,GRP4_SBJ100_A4E0_SMP-blood-sample-3,NO,sample preparation,Unknown,AT11-S3422-raw_data_file-R1
3997,GRP4_SBJ100_A4E0_SMP-blood-sample-3,NO,sample preparation,Unknown,AT11-S3421-raw_data_file-R1
3998,GRP4_SBJ100_A4E1_SMP-blood-sample-1,NO,sample preparation,Unknown,AT11-S3874-raw_data_file-R1


In [23]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt']

,Sample Name,Comment[study step with treatment],Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[library_orientation],Parameter Value[library_strategy],Performer.1,Raw Data File
0,GRP0_SBJ001_A0E0_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S127-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S127-raw_data_file-R1.raw
1,GRP0_SBJ001_A0E0_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S127-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,paired,WGS,Unknown,AT8-S127-raw_data_file-R3.raw
2,GRP0_SBJ001_A0E0_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S127-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S127-raw_data_file-R2.raw
3,GRP0_SBJ001_A0E0_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S127-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,paired,WGS,Unknown,AT8-S127-raw_data_file-R4.raw
4,GRP0_SBJ001_A0E0_SMP-blood-sample-2,NO,extraction,Unknown,AT8-S128-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S128-raw_data_file-R1.raw
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,GRP4_SBJ100_A4E0_SMP-blood-sample-3,NO,extraction,Unknown,AT8-S1711-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,paired,WGS,Unknown,AT8-S1711-raw_data_file-R4.raw
7996,GRP4_SBJ100_A4E1_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S1937-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S1937-raw_data_file-R1.raw
7997,GRP4_SBJ100_A4E1_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S1937-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S1937-raw_data_file-R2.raw
7998,GRP4_SBJ100_A4E1_SMP-blood-sample-1,NO,extraction,Unknown,AT8-S1937-Extract-R1,DNA,library_preparation,Illumina NovaSeq 6000,paired,WGS,Unknown,AT8-S1937-raw_data_file-R4.raw


### Overview of the Clinical Chemistry Marker Panel

For the mass. spec. assay table, we have 160 (derma) samples, 160 extracts (1 per  sample, "polar" fraction), 320 labeled extracts (2 per extract, as "#replicates" is  2) and 640 mass spec protocols + 640 output files (2 per labeled extract as we do 1 technical replicate with 2 protocol parameter combinations `["Agilent QTQF 6510", "FIA", "positive mode"]` and `["Agilent QTQF 6510", "LC", "positive mode"]`).

In [24]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt'].nunique(axis=0, dropna=True)

Sample Name                           2000
Comment[study step with treatment]       1
Protocol REF                             1
Performer                                1
Raw Data File                         4000
dtype: int64

### Overview of the Genome Sequencing assay table

For the NMR assay table, we have 360 (blood) samples, 720 extracts (2 per  sample, a single replicate of the "supernatant" and "pellet" fractions) and 2880 NMR protocols + 4800 output files (4 per extract as we do 2 technical replicates with 2 protocol parameter combinations `["Bruker Avance II 1 GHz", "1D 1H NMR", "CPGM"]` and `["Bruker Avance II 1 GHz", "1D 1H NMR", "TOCSY"]`).

In [25]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'].nunique(axis=0, dropna=True)

Sample Name                             2000
Comment[study step with treatment]         1
Protocol REF                               1
Performer                                  1
Extract Name                            2000
Characteristics[extract type]              1
Protocol REF.1                             1
Parameter Value[instrument]                1
Parameter Value[library_orientation]       2
Parameter Value[library_strategy]          1
Performer.1                                1
Raw Data File                           8000
dtype: int64